In [21]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import plotly as plt

from datetime import datetime
from datetime import timedelta
import plotly.express as px

In [23]:
partners = pd.read_csv('./drive/MyDrive/cmf/partners_delays.csv')
orders = pd.read_csv('./drive/MyDrive/cmf/orders.csv')

In [24]:
#для формирования единого df из двух файлов
def merging(partners, orders):
  #переименуем столбец для мерджа
  orders = orders.rename(columns={"date": "dttm"})
  #объеденим датасеты
  result = pd.merge(partners, orders, how="outer", on=["delivery_area_id", "dttm"])
  #поменяем форматы данных, заменим NaN на 0
  result['orders_cnt'] = result['orders_cnt'].fillna(0)
  result['partners_cnt'] = result['partners_cnt'].astype(int)
  result['orders_cnt'] = result['orders_cnt'].astype(int)
  #вытащим отдельно дату и время
  result[['date', 'time']] = result['dttm'].str.split(' ', expand=True)
  return(result)

#посчитаем количество дней работы сервиса в каждой зоне
def add_days_count(df):
  df1 = df[['delivery_area_id', 'date']].drop_duplicates().reset_index().drop(columns=['index']).drop_duplicates()
  df1['num_days_of_service'] = df1.groupby('delivery_area_id')['date'].transform('count').to_frame()
  df1 = df1[['delivery_area_id', 'num_days_of_service']].drop_duplicates()
  df = pd.merge(df, df1, how="left", on=["delivery_area_id"])
  df['num_days_of_service'] = df['num_days_of_service'].astype(int)
  return(df)

result = merging(partners, orders)
#result = add_days_count(result)
result.head()

,delivery_area_id,dttm,partners_cnt,delay_rate,orders_cnt,date,time
0,0,2021-04-01 09:00:00,3,0.000000,0,2021-04-01,09:00:00
1,0,2021-04-01 10:00:00,4,0.111111,9,2021-04-01,10:00:00
2,0,2021-04-01 11:00:00,4,0.000000,1,2021-04-01,11:00:00
3,0,2021-04-01 12:00:00,4,0.000000,0,2021-04-01,12:00:00
4,0,2021-04-01 13:00:00,1,0.000000,1,2021-04-01,13:00:00


In [25]:
#всего городов = 593
#уникальных дней = 244
#минимальная дата = 2021-04-01 но она разная у всех городов
#макс дата = 2021-11-30 у всех зон

In [26]:
#надо выбрать, например, 3-4 (num_of_cities) крупных города(просто возьмите по наибольшему числу заказов последней даты обуч выборки) 
#и вывести для них массив с днями (num_of_peaks), у которых наибольшая дельта с предшествующими(в процентаже сначала, затем отдельная реализация с абсолютными значениями)

#выбираем количество городов, для которых вывести максимальные отклонения заказов на последнюю дату
def df_with_most_cnt_orders_in_last_day(df, num_of_cities=5):
  cities = df[['delivery_area_id', 'orders_cnt']].loc[df['date']=='2021-11-30']
  cities['sum_of_orders_in_last_date'] = cities.groupby(['delivery_area_id'])['orders_cnt'].transform('sum')
  cities = cities.drop(columns=['orders_cnt']).drop_duplicates().sort_values(by=['sum_of_orders_in_last_date'], ascending=False).reset_index().drop(columns=['index'])
  return(cities.head(num_of_cities))

#для каждого города вытащим количество дней с заданным кол-вом отклонений
def get_days_with_N_highest_peaks(df, num_of_cities=5, num_of_peaks=3):
  x = pd.merge(result, df_with_most_cnt_orders_in_last_day(result, num_of_cities), how="inner", on=["delivery_area_id"]).reset_index()
  x = x[['delivery_area_id', 'date', 'orders_cnt']].groupby(['delivery_area_id', 'date'])['orders_cnt'].sum().reset_index()
  x['shift_orders'] = x.groupby(['delivery_area_id'])['orders_cnt'].shift(1)
  x['delta'] = x['orders_cnt'] - x['shift_orders'] 
  x = x.fillna(0)
  x['delta_pers'] = ((x['orders_cnt'] / x['shift_orders'])*100).round(2)
  for i in range(len(x)):
    if x['delta_pers'][i] == np.inf:
      x['delta_pers'][i] = 100
  for i in range(len(x['delta'])):
    if x['delta'][i] < 0:
      x['delta'][i] = x['delta'][i] * (-1)
  df_with_peak_days = pd.DataFrame()
  for i in x['delivery_area_id'].unique():
    df_with_peak_days = pd.concat([df_with_peak_days, x.loc[x['delivery_area_id'] == i].sort_values(by=['delta'], ascending=False).head(num_of_peaks)])
  df_with_peak_days = df_with_peak_days.reset_index().drop(columns=['index'])
  return(df_with_peak_days)

#вытащить кол-во городов и кол-во пиков для них
def peaks_for_cities(df, num_of_cities, num_of_peaks):
  peaks = get_days_with_N_highest_peaks(df_with_most_cnt_orders_in_last_day(result, num_of_cities=num_of_cities), num_of_cities=num_of_cities, num_of_peaks=num_of_peaks)
  return(peaks)


In [28]:
#вывести три предыдущих значения, затем выброс, затем следующие 3 
#для каждого пика в каждом городе выведем 3 предыдущих дня и 3 следующих дня
def get_days_around_peaks(x):
  x['days_before_3'] = np.nan
  x['days_before_2'] = np.nan
  x['days_before_1'] = np.nan
  x['days_after_3'] = np.nan
  x['days_after_2'] = np.nan
  x['days_after_1'] = np.nan

  list_of_cols = ['days_before_3', 'days_before_2', 'days_before_1', 'days_after_3', 'days_after_2', 'days_after_1']

  for i in range(len(x)):
    x['days_before_3'][i] = str(datetime.strptime(x['date'][i], '%Y-%m-%d') - timedelta(days=3)).split(' ')[0]
  for i in range(len(x)):
    x['days_before_2'][i] = str(datetime.strptime(x['date'][i], '%Y-%m-%d') - timedelta(days=2)).split(' ')[0]
  for i in range(len(x)):
    x['days_before_1'][i] = str(datetime.strptime(x['date'][i], '%Y-%m-%d') - timedelta(days=1)).split(' ')[0]

  for i in range(len(x)):
    x['days_after_3'][i] = str(datetime.strptime(x['date'][i], '%Y-%m-%d') + timedelta(days=3)).split(' ')[0]
  for i in range(len(x)):
    x['days_after_2'][i] = str(datetime.strptime(x['date'][i], '%Y-%m-%d') + timedelta(days=2)).split(' ')[0]
  for i in range(len(x)):
    x['days_after_1'][i] = str(datetime.strptime(x['date'][i], '%Y-%m-%d') + timedelta(days=1)).split(' ')[0]

  x_merged = pd.DataFrame()

  for i in range(len(x)):
    for col in list_of_cols:
      x_merged = pd.concat([x_merged, pd.merge(result, x.iloc[[i]][['delivery_area_id', col]].rename(columns={col: "date"}), how="inner", on=["delivery_area_id", 'date']).groupby(['delivery_area_id', 'date'])['orders_cnt'].sum().to_frame().reset_index()])

  x_merged = pd.concat([x_merged, x[['delivery_area_id', 'date', 'orders_cnt']]]).drop_duplicates().sort_values(by=['delivery_area_id', 'date'], ascending=[False, False]).reset_index().drop(columns=['index'])

  return(x_merged)

In [41]:
#df с пиками по городам (указываем количество городов, выводим заданное кол-во пиков)
peaks = peaks_for_cities(result, num_of_cities=50, num_of_peaks=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
peaks.head()

,delivery_area_id,date,orders_cnt,shift_orders,delta,delta_pers
0,0,2021-05-01,11,55.0,44.0,20.00
1,0,2021-04-19,36,76.0,40.0,47.37
2,0,2021-10-25,88,126.0,38.0,69.84
3,0,2021-11-07,138,101.0,37.0,136.63
4,0,2021-10-28,111,75.0,36.0,148.00


In [ ]:
#присвоим каждой дате номер дня недели
def day_of_week(df):
  df['date_of_week'] = np.nan
  for i in range(len(df)):
    df['date_of_week'][i] = datetime.isoweekday(datetime.strptime(df['date'][i], '%Y-%m-%d').date())
  return(df)

#for i in range(len(raspr)):
#  if raspr['date_of_week'][i] != 1:
#    raspr['date_of_week'][i] = 2
  #peaks['date_of_week'][i] = datetime.isoweekday(datetime.strptime(peaks['date'][i], '%Y-%m-%d').date())

raspr = day_of_week(get_days_around_peaks(peaks))[['date', 'date_of_week']].value_counts().to_frame().reset_index().rename(columns={0: "num_peaks"})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is tryi

In [ ]:
fig = px.histogram(raspr, x=raspr['date'], y=raspr['num_peaks'], nbins=400, color='date_of_week')
fig.show()

In [ ]:

#отдельно вывести максимальный и минимальный матож(в данном случае просто среднее значение) опоздания по всем городам
#написать общий код где будет идти разбивка по размеру города
#найти метод сопоставления дня недели с числом в датасете, думаю, это несложно
#среднее квадратичное отклонение заказов одного города в один день недели и другой
#матож количества заказов и наибольшее, тут очень много зон где один заказ в час


#макс значения дилея
#дисперсия набора коэффициентов дилей рейт отклонений
#пики дней искать по дельте 3 дня назад 3 дня вперед
#посмотреть пересечения пиков по большим городам
#метод разбивает массивы по признакам
#функция которая строит массив по пикам дельты
#вызвали для 100 остальных больших городов
#построили массив или двойной массив
#прогоняем массив где сопостовляем число с другими городами
#записываем в массив с пиками если пики больше какого то числа то записываем дату в массив
#сет из ста массивов

,delivery_area_id,date,partners_cnt,delay_rate,orders_cnt
0,0,2021-04-01,26,0.111111,24
1,0,2021-04-02,20,0.000000,21
2,0,2021-04-03,22,0.583333,33
3,0,2021-04-04,20,0.000000,18
4,0,2021-04-05,24,0.166667,32
...,...,...,...,...,...
99332,592,2021-11-26,42,0.312500,56
99333,592,2021-11-27,30,0.652778,50
99334,592,2021-11-28,25,0.250000,46
99335,592,2021-11-29,47,0.900000,73
